## Glass training
 This notebook loads the Glass identification dataset (https://archive.ics.uci.edu/dataset/42/glass+identification), preprocesses it and trains a simple model

In [1]:
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))

import xai_faithfulness_experiments_lib_edits as fl

In [2]:
import numpy as np

file_data = np.load(os.path.join(PROJ_DIR, 'assets', 'data', 'avila.npz'))
x_train = file_data['x_train']
x_test = file_data['x_test']
y_train = file_data['y_train']
y_test = file_data['y_test']

Load data from file

In [3]:
import torch
class RandomMasker(torch.nn.Module):
    def __init__(self, masking_value:torch.tensor):
        super().__init__()
        self._masking_value = masking_value
    def forward(self, x): # Assumes inputs are (batch_size, num_vars)
        selection_levels = torch.rand((x.shape[0], 1)) # A different selection level for each element of the batch
        selected_pixels = torch.le(torch.rand(x.shape), selection_levels) # A different selection level for each element of the batch
        return x * selected_pixels + self._masking_value * ~selected_pixels

# Zeros
masking_value = np.zeros(x_train.shape[1])
# Mean
masking_value = np.mean(x_train, axis=0)
masker = RandomMasker(torch.tensor(masking_value).float())

Train a MLP model

In [6]:
MODEL_NEURONS = 100
MODEL_EPOCHS= 2000
MODEL_LR = 1.0e-1
MODEL_LABEL_NUM = len(np.unique(y_train))

x_train_tensor = torch.tensor(x_train).float()
y_train_tensor = torch.tensor(y_train)
x_test_tensor = torch.tensor(x_test).float()
y_test_tensor = torch.tensor(y_test)

label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
class_weights = 1.0/label_onehot.mean(axis=0)

network = fl.MLP(x_train.shape[1], MODEL_LABEL_NUM, MODEL_NEURONS)
loss = torch.nn.BCELoss(weight=class_weights)
optimizer = torch.optim.Adam(network.parameters(), lr=MODEL_LR)#, weight_decay=1e-3)

MODEL_EPOCHS = 0 # Untrained
for epoch in range(MODEL_EPOCHS):
    x_train_tensor_masked = masker(torch.tensor(x_train).float()) # A different set of RandomMasks for each batch
    optimizer.zero_grad()
    
    preds = network(x_train_tensor_masked)
    label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
    label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
    loss_value = loss(preds, label_onehot)
    loss_value.backward()        
    optimizer.step()

    train_accuracy = (preds.argmax(dim=1) == y_train_tensor).float().mean() 

    test_preds = network.forward(x_test_tensor)        
    test_accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
    print(f'Epoch {epoch}/{MODEL_EPOCHS} - Loss: {loss_value.item()} - Train accuracy: {train_accuracy} - Test accuracy: {test_accuracy}')  
    #if test_accuracy > 0.6: # Undertrained
    #    break

network.eval()
test_preds = network.forward(x_test_tensor)        
accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean()
print(accuracy.item())


0.7996165156364441


In [7]:
# Save model
torch.save(network.state_dict(), os.path.join(PROJ_DIR,'assets','models','avila-ood-mean-mlp.pth'))

import json
MODELS_PATH = os.path.join(PROJ_DIR,'assets','models')
with open(os.path.join(MODELS_PATH, 'model-accuracies.json')) as fIn:
    models = json.load(fIn)
models['avila-ood-mean'] = accuracy.item()
with open(os.path.join(MODELS_PATH, 'model-accuracies.json'), 'w') as fOut:
    json.dump(models, fOut)